# <center> Классификация рукописных цифр
## <center> MNIST
<img src='./img/mnist_examples.png'>

## Метод ближайших соседей

In [ ]:
# импортируем требуемые библиотеки
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from source.mnistloader import get_mnist_data
import pickle

In [ ]:
# загрузим датасет
X_train, y_train = get_mnist_data(train=True)
X_test, y_test = get_mnist_data(train=False)

In [ ]:
# выведем размер имеющейся выборки
print('Размер обучающей выборки: %d изображений' % len(X_train))
print('Размер тестовой выборки: %d изображений' % len(X_test))

In [ ]:
# Создадим модель классификатора.
# Параметры модели:
# 
#   n_neighbors : количество ближайших соседей, по которым будем определять класс
#   weights     : алгоритм, по которому учитывается "голос" каждого из соседей
#      - 'uniform' : веса одинаковы
#      - 'distance' : веса обратно-пропорциональны расстоянию до соседа

knn_model = KNeighborsClassifier(n_neighbors=5, 
                                 weights='uniform')

In [ ]:
%%time

# Обучим KNN-модель, засекая время с помощью функции %%time
knn_model.fit(X_train, y_train)

In [ ]:
%%time

# проверим процент точности на тестовой выборке
y_pred = knn_model.predict(X_test[:1000])
(y_pred == y_test[:1000]).mean()

In [ ]:
%%time

# посмотрим сколько времени требуется для того, чтобы сделать одно предсказание
knn_model.predict(X_test[:1]);

In [ ]:
# сохраним модель для последующего тестирования
with open('knn.pickle', 'wb') as out:
    pickle.dump(knn_model, out)

## Сверточная нейросеть

In [ ]:
# импортируем требуемые библиотеки
import torch
import matplotlib.pyplot as plt
from source.torchutils import ModelWithAPI, Flatten, Softmax
from source.mnistloader import create_mnist_loader
from source.utils import plot_grid
from albumentations import Compose, HorizontalFlip, Cutout, ShiftScaleRotate, ToGray, ToFloat, Transpose, PadIfNeeded, Resize
from albumentations.pytorch import ToTensor
import PIL
import cv2
import numpy as np

In [ ]:
# создадим загрузчики данных
# тут мы можем регулировать размер обучающей и тестовой выборки (параметр size можно менять)

train_loader = create_mnist_loader(size=0.3)

test_loader = create_mnist_loader(size=0.2,
                                  train=False)

In [ ]:
# посмотрим как будут выглядят данные каждого класса
plot_grid(train_loader.dataset.data, train_loader.dataset.targets, 
          class_num = 2,  # - какой класс показать 
          grid_size = 10) # - размер сетки

In [ ]:
# определим архитектуру нейросети

conv_net = torch.nn.Sequential(torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=3),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(num_features=16),
                               # 16x16x16
                               
                               torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(32),
                               # 8x8x32
                               
                               torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 4x4x64
                               
                               torch.nn.Conv2d(64, 128, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 2x2x64
                               
                               Flatten(),
                               torch.nn.Linear(512, 10), 
                               Softmax())

# определим правило, по которому будем менять веса в нейросети
optim = torch.optim.Adam(conv_net.parameters(), 
                         5e-3) # - шаг обучения

# выберем функцию ошибки
criterion = torch.nn.functional.cross_entropy

In [ ]:
# соберем нашу модель
model = ModelWithAPI(conv_net, optim, criterion, device='cpu')

In [ ]:
# и обучим ее
train_history, val_history = model.fit_loader(train_loader,
                                              val_loader=test_loader,
                                              epochs=30,              # - количество эпох
                                              lr_decay_every=10);     # - через сколько эпох будем уменьшать шаг

In [ ]:
%%time

# посмотрим на процент правильно угаданных ответов в тестовой выборке, замерив время
pred = model.evaluate_metrics_loader(test_loader)
(pred == test_loader.dataset.targets).mean()

In [ ]:
# сохраним нашу модель
torch.save(model.model, './cnn_model.pt') 

## Сверточная нейросеть с аугментацией

In [ ]:
# определим какие методы аугментации, которы будем использовать
aug_trs = Compose([PadIfNeeded(45, 45, border_mode=0, p=0.2),
                   
                   ShiftScaleRotate(           # - случайное смещение, масштабирование, повороты
                       shift_limit= 0.2,       # - максимальное смещение (в долях 0.2 - это на 20% в сторону)
                       scale_limit=(0, 0.5),   # - границы масштабирования
                       rotate_limit=10,        # - максимальный угол поворота
                       border_mode=0,          # - лучше оставить как есть :)
                       p=0.2),                 # - вероятность, с которой будет применена аугментация
                   
                  Resize(28, 28, always_apply=True),
                   
                   Cutout(                     # - случайно расбрасываем на картинке прямоугольники
                       num_holes=16,           # - кол-во прямоугольников
                       max_h_size=3,           # - высота прямоугольника
                       max_w_size=3,           # - ширина
                       fill_value=0,           # - яркость прямоугольников
                       p=0.5),                 # - вероятность, с которой будет применена аугментация
                   

                   Transpose(always_apply=True),
                   ToTensor()])

In [ ]:
# создадим загрузчик тренировочных данных, указав аугментацию
# тут мы можем регулировать размер обучающей и тестовой выборки (параметр size можно менять)
train_loader = create_mnist_loader(size=0.3, 
                                   trs = aug_trs)

In [ ]:
# посмотрим как будут выглядеть наши данные после применения аугментации
plot_grid(train_loader.dataset.data, train_loader.dataset.targets, 
          trs = aug_trs,
          class_num = 2,  # - какой класс показать 
          grid_size = 10) # - размер сетки

In [ ]:
# снова определим архитектуру нейросети

conv_net = torch.nn.Sequential(torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=3),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(num_features=16),
                               # 16x16x16
                               
                               torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(32),
                               # 8x8x32
                               
                               torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 4x4x64
                               
                               torch.nn.Conv2d(64, 128, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 2x2x64
                               
                               Flatten(),
                               torch.nn.Linear(512, 10), 
                               Softmax())

# определим правило, по которому будем менять веса в нейросети
optim = torch.optim.Adam(conv_net.parameters(), 
                         5e-3) # - шаг обучения

# выберем функцию ошибки
criterion = torch.nn.functional.cross_entropy

In [ ]:
# соберем нашу модель
model = ModelWithAPI(conv_net, optim, criterion, device='cpu')

In [ ]:
# и обучим ее
train_history, val_history = model.fit_loader(train_loader,
                                              val_loader=test_loader,
                                              epochs=30,              # - количество эпох
                                              lr_decay_every=10);     # - через сколько эпох будем уменьшать шаг

In [ ]:
%%time

# посмотрим на процент правильно угаданных ответов в тестовой выборке, замерив время
pred = model.evaluate_metrics_loader(test_loader)
(pred == test_loader.dataset.targets).mean()

In [ ]:
# сохраним нашу модель
torch.save(model.model, './cnn_model_aug.pt')